In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

###### spark 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

###### load clickstream 

In [3]:
clickstream = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_CLICKSTREAM, hive)

In [4]:
clickstream.show(3)

,inn,timestamp,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,product
0,470707811305,1623239609,/accounts-payments,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],
1,6220008542,1623318711,/main,СберБизнес,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: true]",
2,5260170451,1623412583,/accounts-payments/account/:accountId,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],


In [5]:
clickstream.filter(f.isnull("inn")).count()

0

###### get user id 

In [6]:
sbbol_user_id = clickstream.select("inn").distinct().cache()

In [7]:
sbbol_user_id.show(3)

,inn
0,1644011469
1,7728294694
2,524908587394


In [8]:
w = Window().orderBy("inn")

In [9]:
user_dict = sbbol_user_id.withColumn("user_id", f.row_number().over(w))

In [17]:
user_dict = user_dict.filter("inn not in ('00000', '0000000000', '000000000000') ")

###### save user_dict 

In [18]:
drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_USER_DICT, hive)

In [19]:
%%time
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_USER_DICT, user_dict, hive)

CPU times: user 2.89 ms, sys: 3.14 ms, total: 6.03 ms
Wall time: 4.24 s


In [20]:
load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_USER_DICT, hive).count()

1038606